# Visualization of 3D local thickness

All examples below show the local thickness of the small test volume. 

:warning: **The examples are written such that you should run only one of the cells. Restart the kernel when trying different approach. You should be able to find a visualization option which works on your setup.** 

## Approach 1: Use interactive slicer from localthickness

You can use interactive matplotlib figure. However, this depends on how you run python and requires an appropriate matplotlib [backend](https://matplotlib.org/stable/users/explain/backends.html). If you setup supports interactive figures using `Qt5Agg` backend, the code below will start the interactive figure in a new window. You can use keybord keys (`left`/`a`, `right`/`d`, `up`/`w`, `down`/`z`, `pageup`/`e`, and `pagedown`/`x`) to inspect the slices of the volume. Make sure that the window is active (click into the colored part of the figure), and try interactive keys, e.g. `a` and `d`. Some keys might be reserved for other commands (e.g. left brings previous zoom level) so try finding out what works for you.

If the figure does not show, or is not interactive, try some other (interactive) backends as shown below. Remember to restart the kernel when changing the backend. In IPython, you can activate backends using magic commands. 
`` 
matplotlib.use('Qt5Agg')  # Change matplotlib backend to something, here Qt5Agg, also try TkAgg
``

``
%matplotlib qt  # Activate backend in IPython, here qt, try also tk
``

:warning: **You should restar the kernel when moving to other approaches 2 after trying approach 1.**

In [2]:
# visulizing slices of the volume
import matplotlib
import localthickness as lt
import tifffile

matplotlib.use('Qt5Agg')

B = tifffile.imread('data/cotton_test_volume.tif') > 0
T = lt.local_thickness(B, scale=0.25)

lt.show_vol(T, cmap=lt.black_plasma())


## Approach 2: Show slices using ipywidgets

:warning: **You shoul restar the kernel when moving to approach 2 after trying approach 1.**

In [ ]:
import localthickness as lt
import tifffile
from ipywidgets import interact

B = tifffile.imread('data/cotton_test_volume.tif') > 0
T = lt.local_thickness(B, scale=0.25)

interact(lambda z: lt.view_thickness_slice(z, T, T.max()), z=(0, T.shape[0]-1));




## Approach 3: Use plotly for 3D visualization
With [plotly](https://plot.ly/python/) we can see the data and the thickness as an interactive 3D visualization. __Be patient__, tt may take up to a minute for plotly to produce a figure. 

In [ ]:
import localthickness as lt
import tifffile
import plotly.graph_objects as go
import numpy as np

B = tifffile.imread('data/cotton_test_volume.tif') > 0
T = lt.local_thickness(B, scale=0.25)


X, Y, Z = np.mgrid[range(B.shape[0]), range(B.shape[1]), range(B.shape[2])]

fig = go.Figure(data = go.Volume(x=X.flatten(), y=Y.flatten(), z=Z.flatten(), value=T.flatten(),
    isomin=0, isomax=T.max(), opacity=0.1, surface_count=15, colorscale = lt.pl_black_plasma()))
fig.show()

## Approach 4: Save and show in paraview

To show thickness in paraview:
- Load both `*_gray.tif` and `*_rgba.tif` volume. Apply.
- Select **both** volumes, and via right-click menu add filter `Append Attributes`. Apply.
- Select the new data and add filter `Contour`.
- Select contour and change following properties (you might need to click cogwheel icon to access all properties)
  - In **Isosurfaces** adjust the first (only) isovalue to 127.5,
  - In **Coloring** choose Tiff_Scalars_input1 (or Tiff_Scalars, depending on whether you first read rgba volume or gray),
  - In **Scalar Coloring**, turn off Map Scalars.

In [1]:
import localthickness as lt
import tifffile
import plotly.graph_objects as go
import numpy as np

B = tifffile.imread('data/cotton_test_volume.tif') > 0
T = lt.local_thickness(B, scale=0.25)

lt.save_thickness_volumes(T, 'demo_thickness')

The volume should now show as in the screenshot below.

<img src="cotton_vis.png" width="50%" height="50%">


## Other saving options


In [12]:
import localthickness as lt
import tifffile
import plotly.graph_objects as go
import numpy as np

B = tifffile.imread('data/cotton_test_volume.tif') > 0
T = lt.local_thickness(B, scale=0.25)

T = T/T.max()  # Omit this if you need absolute values
rgba = lt.black_plasma()(T)
rgba = (255*rgba).astype(np.uint8)

tifffile.imsave('test_T.tif', T)
tifffile.imsave('test_rgba.tif', rgba, photometric='rgb', extrasamples = 'ASSOCALPHA')
